In [2]:
# Load the path from Valhalla

import requests
import json

def get_distance(pointA, pointB):
    url = "http://valhalla:8002/route"
    data =  {
        "locations":[
            {
                "lat":pointA[1],
                "lon":pointA[0],
                "type":"break"
            },{
                "lat":pointB[1],
                "lon":pointB[0],
                "type":"break"
            }
        ],
        "costing":"pedestrian",
        "directions_options":{
            "units":"miles"
        }
    }

    data_json = json.dumps(data)
    response = requests.post(url, data=data_json)
    response_obj = json.loads(response.text)
    #distance = response_obj['trip']['summary']['length']
    time_in_seconds = response_obj['trip']['summary']['time']
    #west = response_obj['trip']['summary']['min_lon']
    #south = response_obj['trip']['summary']['min_lat']
    #east = response_obj['trip']['summary']['max_lon']
    #north = response_obj['trip']['summary']['max_lat']
    #geometry = decode_geometry(response_obj['trip']['legs'][0]['shape'])
    return time_in_seconds/60

In [17]:
import psycopg2
conn = psycopg2.connect(host="postgres",database="osm", user="osm", password="osm")
cursor = conn.cursor()
lat = 40.222884
lon = -74.010232
distance_in_meters = 2000
amenity = 'restaurant'
cursor.execute("""
select * FROM (
    select
     name,
     ST_X(ST_Transform(way, 4326)) as lon,
     ST_Y(ST_Transform(way, 4326)) as lat,
     ST_DISTANCE(
         way,
         ST_TRANSFORM(
             ST_SETSRID(
                 ST_GeomFromText(
                     'POINT(%f %f)'
                 ),
             4326),
         3857)
     ) AS distance
    FROM
       planet_osm_point
    WHERE
      amenity = '%s'
) AS subquery WHERE distance < %f 
ORDER BY distance;
""" % (lon, lat, amenity, distance_in_meters))
pointA = [lon, lat]
returned_data = []
for record in cursor:
    pointB = [record[1], record[2]]
    time = get_distance(pointA, pointB)
    print (record[0], "(%f min)" % time, record[3])
    returned_data.append(record + (time,))
    
cursor.close()


[('Jonti S Latin & Soul Food', -74.0112807855013, 40.2177291886759, 760.528083688581, 6.9), ('Pascal & Sabine', -74.0102024478343, 40.2170361010522, 852.559714144463, 9.016666666666667), ('Cardinal Provisions', -74.0093831842951, 40.2168760670453, 880.965369533937, 9.9), ('Barrio Costero', -74.0104997003618, 40.2167567101032, 893.780302984483, 9.116666666666667), ('The Bonney Read', -74.0094819989764, 40.2165348702625, 929.381059187401, 10.666666666666666), ("Lola's European Cafe", -74.0097496070995, 40.2164179135915, 944.201640732824, 10.9), ('Cross & Orange', -74.0086790847755, 40.2164337593459, 956.12091627585, 11.066666666666666), ('Asbury Ale House', -74.0100469494586, 40.2163148817998, 957.914888938367, 10.566666666666666), ('Toast Asbury Park', -74.0088406018635, 40.2163919841674, 959.043693106551, 10.9), ('Brickwall Tavern & Dining Room', -74.009011191936, 40.2163384103786, 963.891752771227, 10.7), ("DJ's Delights Deli & Restaurant", -74.0112886906758, 40.2161490153772, 988.894

In [18]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Bring the returned data into a pandas dataframe
df = pd.DataFrame(returned_data, columns = ["Name", "Lon", "Lat", "Distance", "Time"])

# zip the lon and lat into a coordinates column
df['Coordinates'] = list(zip(df.Lon, df.Lat))

# Now we can get rid of those lon and lat columns
df.drop('Lon', axis=1, inplace=True)
df.drop('Lat', axis=1, inplace=True)

# GeoPandas needs a Shapely geometry, so convert that lon,lat to a Point
df['Coordinates'] = df['Coordinates'].apply(Point)

gdf = gpd.GeoDataFrame(df, geometry='Coordinates')


In [28]:
# Draw Path on a leaflet map

from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, Popup
from ipywidgets import HTML
from shapely.geometry import shape, MultiPoint
import json
import math

# This isn't a ipyleaflet function for some reason!
def fitBounds(m, geojson):
    # Center the map on our shape!
    # Draw a buffer around the whole thing
    polygon = MultiPoint(list(map(lambda x: x['geometry']['coordinates'],geojson['features']))).convex_hull
    new_bounds = polygon.buffer(0.001).bounds
    new_center = [(new_bounds[1] + new_bounds[3]) / 2, (new_bounds[0] + new_bounds[2]) / 2]
    new_zoom = math.floor(
            min(math.log(180/(new_bounds[2] - new_bounds[0]), 2), math.log(360/(new_bounds[3] - new_bounds[1]), 2)))

    m.center = new_center
    m.zoom = new_zoom
    return m

m = Map(
    layers=(basemap_to_tiles(basemaps.CartoDB.DarkMatter), )
)

out_data = json.loads(gdf.to_json())
geo_json = GeoJSON(data=out_data)
m.add_layer(geo_json)

m = fitBounds(m, out_data)


m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [29]:
print(gdf)

                                  Name     Distance       Time  \
0            Jonti S Latin & Soul Food   760.528084   6.900000   
1                      Pascal & Sabine   852.559714   9.016667   
2                  Cardinal Provisions   880.965370   9.900000   
3                       Barrio Costero   893.780303   9.116667   
4                      The Bonney Read   929.381059  10.666667   
5                 Lola's European Cafe   944.201641  10.900000   
6                       Cross & Orange   956.120916  11.066667   
7                     Asbury Ale House   957.914889  10.566667   
8                    Toast Asbury Park   959.043693  10.900000   
9       Brickwall Tavern & Dining Room   963.891753  10.700000   
10     DJ's Delights Deli & Restaurant   988.894428   9.566667   
11                   Twisted Tree Cafe   999.968517  10.666667   
12                            Talula's  1002.033265  11.133333   
13                          Moonstruck  1009.622915  12.783333   
14        